In [2]:
import solara
import solara.lab
import ipyleaflet
import ipywidgets as widgets
from IPython.display import display
import json
import datetime as dt
from pathlib import Path



# Step 1: Select scenario

set paths to folders
Perhaps in demonstrator no need to select paths but directly the data sets


## select FoodAdapt data base for which we are configuring scenarios

In [3]:
# HERE: DROP-DOWN MENU; people won't have datasets on their local drive

FA_databases = ["Scheveningen", "Option B", "Option C"] #################################
FA_database = solara.reactive("Scheveningen")


@solara.component
def Page():
    solara.Select(label="FloodAdapt data base", value=FA_database, values=FA_databases)
    
    solara.Markdown(f"**Selected**: {FA_database.value}")


########################################################################################################################################################################################
# The following line is required only when running the code in a Jupyter notebook:
Page()

Cannot show ipywidgets in text

According to the selection above, the path to the FloodAdapt database is set below. Needs adjustment! ##########################

In [5]:
if FA_database.value == "Scheveningen":
    fa_database = Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/scheveningen")
elif FA_database.value == "Option B":
    fa_database = Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/OPTIONB/") #########################
else:
    fa_database = Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/OPTIONC/") #########################

# print(fa_database)

## Select (path to) data

In [6]:
# HERE: whatever the solutoin is; people won't have datasets on their local drive

# Path to data folder. If (relative) file paths are used when specifying data sets, it will be looked for in this folder.
# data_folder = Path("/home/wotromp/InterTwin/Data")                              #########################


## Select HydroMT data catalogue from which to fetch data


In [7]:
# OPTIONAL: HydroMT data catalogue from which to fetch data. The order matters here, as datasets are looked for in a catalogue in order from first catalogue name in list to last.
data_catalogues = [Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/deltares_data_wsl.yml")]

# Step 2: Configure the weather event

we need: 

-  Name for referencing
-  start and end time and dates
-  forcing data for hazard model [...]

## Event Name

In [8]:
event_name_input = solara.reactive("Event name")

@solara.component
def Page():
    solara.InputText("Name of the event", value=event_name_input)#, continuous_update=continuous_update.value)
    # with solara.Row():
    #    solara.Button("Clear", on_click=lambda: text.set(""))
    solara.Markdown(f"**Your name of the event**: {event_name_input.value}")


########################################################################################################################################################################################
# The following line is required only when running the code in a Jupyter notebook:
Page()

Cannot show ipywidgets in text

In [7]:
# in case the input has spaces, replace them by "_"

def replace_spaces_with_underscores(string):
    if ' ' in string:
        new_string = string.replace(' ', '_')
    else:
        new_string = string
    
    return new_string

event_name = replace_spaces_with_underscores(str(event_name_input))

print(event_name_input)
print(event_name)

'Event name'
'Event_name'


## Start and End time

Set Start and End time of the event

### in Solara (pick date, needs limitations)

limitation not possible with datepicker, perhaps drop-down menu to select year, then month, then day. this way limitation possible

In [8]:
# select start date

@solara.component
def Page():
    date = solara.use_reactive(dt.date.today())

    
    solara.lab.InputDate(date)
    solara.Text(str(date.value))



########################################################################################################################################################################################
# The following line is required only when running the code in a Jupyter notebook:
Page()

Cannot show ipywidgets in text

selecting date and time works well with ipywidgets, but not with solara somehow (yet)

### Via ipywidgets: pick date and time but ugly design

Select start date & time

In [9]:
# Create date and time pickers
start_date = widgets.DatePicker(
    description='Select Date',
    value=dt.date.today()
)
start_time = widgets.Text(
    description='Select Time',
    value=dt.datetime.now().strftime('%H:%M'),
    placeholder='HH:MM'
)

# Create a button to update the selected datetime
update_button = widgets.Button(description="Save Start Date")

# Output area for displaying the selected datetime
output = widgets.Output()

# Define the function to be called when the button is clicked
def on_button_click(b):
    # Combine date and time into a single datetime object
    date_value = start_date.value
    time_value = start_time.value
    try:
        time_obj = dt.datetime.strptime(time_value, '%H:%M').time()
        selected_datetime = dt.datetime.combine(date_value, time_obj)
        formatted_datetime = selected_datetime.strftime('%Y-%m-%d %H:%M:%S')
    except ValueError:
        formatted_datetime = "Invalid time format. Use HH:MM."
    
    # Display the selected datetime
    with output:
        output.clear_output()
        print(f"Selected DateTime: {formatted_datetime}")

# Attach the button click event
update_button.on_click(on_button_click)

# Display widgets
display(start_date, start_time, update_button, output)

DatePicker(value=datetime.date(2024, 8, 1), description='Select Date', step=1)

Text(value='13:27', description='Select Time', placeholder='HH:MM')

Button(description='Save Start Date', style=ButtonStyle())

Output()

Select end date & time

In [10]:
# Create date and time pickers
end_date = widgets.DatePicker(
    description='Select Date',
    value=dt.date.today()
)
end_time = widgets.Text(
    description='Select Time',
    value=dt.datetime.now().strftime('%H:%M'),
    placeholder='HH:MM'
)

# Create a button to update the selected datetime
update_button = widgets.Button(description="Save End Date")

# Output area for displaying the selected datetime
output = widgets.Output()

# Define the function to be called when the button is clicked
def on_button_click(b):
    # Combine date and time into a single datetime object
    date_value = end_date.value
    time_value = end_time.value
    try:
        time_obj = dt.datetime.strptime(time_value, '%H:%M').time()
        selected_datetime = dt.datetime.combine(date_value, time_obj)
        formatted_datetime = selected_datetime.strftime('%Y-%m-%d %H:%M:%S')
    except ValueError:
        formatted_datetime = "Invalid time format. Use HH:MM."
    
    # Display the selected datetime
    with output:
        output.clear_output()
        print(f"Selected DateTime: {formatted_datetime}")

# Attach the button click event
update_button.on_click(on_button_click)

# Display widgets
display(end_date, end_time, update_button, output)

DatePicker(value=datetime.date(2024, 8, 1), description='Select Date', step=1)

Text(value='13:27', description='Select Time', placeholder='HH:MM')

Button(description='Save End Date', style=ButtonStyle())

Output()

In [11]:
# start_time.value
# end_date.value

print('Start time: ' + str(start_date.value)+'   '+str(start_time.value))
print('End time:   ' + str(end_date.value)+'   '+str(end_time.value))

Start time: 2024-08-01   13:27
End time:   2024-08-01   13:27


## Forcing data for the SFINCS model

### Meteorological data:

In [12]:
# HERE: DROP-DOWN MENU

met_datas = ["ERA5 Hourly", "Option B", "Option C"] #################################
met_data = solara.reactive("ERA5 Hourly")


@solara.component
def Page():
    solara.Select(label="Meteorological data", value=met_data, values=met_datas)
    solara.Markdown(f"**Selected**: {met_data.value}")


########################################################################################################################################################################################
# The following line is required only when running the code in a Jupyter notebook:
Page()

Cannot show ipywidgets in text

In [13]:
if met_data.value == "ERA5 Hourly":
    sfincs_meteo = 'era5_hourly'
elif met_data.value == "OPTION B": ############################################
    sfincs_meteo = 'OPTION_B' ############################################
else:                         ############################################
    sfincs_meteo = 'OPTION_C' ############################################

print(sfincs_meteo)



era5_hourly


### Waterlevel data:

In [14]:
# HERE: DROP-DOWN MENU

wl_datas = ["gtsm_reanalysis_hourly", "Option B", "Option C"] #################################
wl_data = solara.reactive("gtsm_reanalysis_hourly")


@solara.component
def Page():
    solara.Select(label="Waterlevel", value=wl_data, values=wl_datas)
    solara.Markdown(f"**Selected**: {wl_data.value}")


########################################################################################################################################################################################
# The following line is required only when running the code in a Jupyter notebook:
Page()

Cannot show ipywidgets in text

In [15]:
if wl_data.value == "gtsm_reanalysis_hourly":
    sfincs_waterlevel = 'gtsm_reanalysis_hourly'
elif wl_data.value == "OPTION B": ############################################
    sfincs_waterlevel = 'OPTION_B' ############################################
else:                         ############################################
    sfincs_waterlevel = 'OPTION_C' ############################################

print(sfincs_waterlevel)

gtsm_reanalysis_hourly


## Define event_dict


In [16]:

event_dict = {
    'name': event_name,
    'start_time': start_time,
    'end_time': end_time,
    "data_catalogues":  [str(path) for path in data_catalogues],
    'sfincs_forcing': {
        'meteo': sfincs_meteo,
        'waterlevel': sfincs_waterlevel
    },
}

# Step 3: Configure projections

Physical Projections:
-  Sea level rise
-  Land subsidence
-  Rainfall increase
-  Storm frequency increase

Socio-Economic Projections:
-  Population growth
-  Economic growth
-  New developments:
    -  Newly developed area
    -  Elevation of a newly developed area
    -  Population growth in newly developed area

## Name of Projection

In [17]:
projection_name_input = solara.reactive("Projection name")

@solara.component
def Page():
    solara.InputText("Name of the projection", value=projection_name_input)#, continuous_update=continuous_update.value)
    # with solara.Row():
    #    solara.Button("Clear", on_click=lambda: text.set(""))
    solara.Markdown(f"**Your name of the projection**: {projection_name_input.value}")


########################################################################################################################################################################################
# The following line is required only when running the code in a Jupyter notebook:
Page()

Cannot show ipywidgets in text

In [18]:
# in case the input has spaces, replace them by "_"

projection_name = replace_spaces_with_underscores(str(projection_name_input))

print(projection_name_input)
print(projection_name)

'Projection name'
'Projection_name'


## Physical projections

### Sea Level Rise  [m]

Slider has the advantage that the user has directly a realistic scale, while when leaving it open, a non-expert user might put maybe 10 m sealevel rise

In [19]:
sea_level_rise = solara.reactive(0)

@solara.component
def Page():
    solara.SliderFloat("Sea Level Rise [m]", value=sea_level_rise, min=0, max=2) #, step=0.01)
    solara.Markdown(f"**Sea Level Rise [m]**: {sea_level_rise.value}")
    with solara.Row():
        solara.Button("Reset", on_click=lambda: sea_level_rise.set(0))

########################################################################################################################################################################################
# The following line is required only when running the code in a Jupyter notebook:
Page()

Cannot show ipywidgets in text

### Rainfall Increase  [\%] (whatever realistic)
chose where to limit the user in selection of the value in %

In [20]:
rainfall_increase = solara.reactive(0)

@solara.component
def Page():
    solara.SliderFloat("Rainfall Increase [%]", value=rainfall_increase, min=0, max=33.3) #, step=0.01)
    solara.Markdown(f"**Rainfall Increase [%]**: {rainfall_increase.value}")
    with solara.Row():
        solara.Button("Reset", on_click=lambda: rainfall_increase.set(0))


########################################################################################################################################################################################
# The following line is required only when running the code in a Jupyter notebook:
Page()

Cannot show ipywidgets in text

## Socio-Economic Projections


### Population Growth  [\%] (whatever realistic)
chose where to limit the user in selection of the value in %

In [21]:
pop_growth = solara.reactive(0)

@solara.component
def Page():
    solara.SliderFloat("Population Increase [%]", value=pop_growth, min=0, max=33.3) #, step=0.01)
    solara.Markdown(f"**Population Increase [%]**: {pop_growth.value}")
    with solara.Row():
        solara.Button("Reset", on_click=lambda: pop_growth.set(0))


########################################################################################################################################################################################
# The following line is required only when running the code in a Jupyter notebook:
Page()

Cannot show ipywidgets in text

### Economic Growth [\%] 

In [22]:
economic_growth = solara.reactive(0)

@solara.component
def Page():
    solara.SliderFloat("Economic Increase [%]", value=economic_growth, min=0, max=33.3) #, step=0.01)
    solara.Markdown(f"**Economic Increase [%]**: {economic_growth.value}")
    with solara.Row():
        solara.Button("Reset", on_click=lambda: economic_growth.set(0))


########################################################################################################################################################################################
# The following line is required only when running the code in a Jupyter notebook:
Page()

Cannot show ipywidgets in text

## Define proj_dict 


In [23]:
SLR = sea_level_rise.value
RFI = rainfall_increase.value
PG = pop_growth.value
EG = economic_growth.value

proj_dict = {
    'name': projection_name,
    'physical_projection': {
        'sea_level_rise': SLR,
        'rainfall_increase': RFI
    },
    'socio_economic_change': {
        'population_growth_existing': PG,
        'economic_growth': EG
    }
}

# Step 4: Configure Measures

Hazard Type:
-  Floodwall
-  Pump
-  Water square
-  Green infrastructure
-  Water storage

Impact Type:
-  Elevate properties
-  Floodproof properties
-  Buyout properties


## Reset measures to include in the scenario

### Strategy name

## Single measures
Rerun this cell multiple times to include multiple measures, everytime with updated variables appropriate to the to-be-added measures

### Name of the measure

### Type of the measure

### >>These settings depend on the measure, already in a FloodAdapt format<<

### Update strategy/measure

# play around map and try to draw something in there

In [26]:
# Reactive variable to store drawn polygons
drawn_polygons = solara.reactive([])

@solara.component
def Page():
    # Create a map centered at a specific location
    m = ipyleaflet.Map(
        center=(52.08654741528378, 4.295223531699989),
        zoom=10,
        scroll_wheel_zoom=True
    )
    
    # Create a DrawControl instance
    draw_control = ipyleaflet.DrawControl(
        polyline={'shapeOptions': {'color': 'blue', 'weight': 4}},  # Configuration for polylines
        polygon={'shapeOptions': {'color': 'red', 'weight': 4}},  # Configuration for polygons
        # circle={'shapeOptions': {'color': 'green', 'weight': 4}},  # Configuration for circles
        # rectangle={'shapeOptions': {'color': 'purple', 'weight': 4}},  # Configuration for rectangles
        marker={}  # Configuration for markers
    )
    
    # Function to handle drawing events
    def handle_draw(target, action, geo_json):
        # print("geo_json received:", geo_json)
        if action == 'created':

            if 'geometry' in geo_json:
                geom_type = geo_json['geometry']['type']
                coords = geo_json['geometry']['coordinates']
                
                # Append the drawn feature to the reactive variable
                drawn_polygons.value.append({'type': geom_type, 'coordinates': coords})
                print(f"Type: {geom_type}")
                print(f"Coordinates: {coords}")
            else:
                print("Unexpected geo_json structure:", geo_json)

    # Attach the draw control to the map
    m.add_control(draw_control)
    
    # Set up event listener for drawing actions
    draw_control.on_draw(handle_draw)
    
    # Create a VBox to contain both the map and the information display
    map_widget = widgets.Output()
    
    # Use ipywidgets to display the map
    with map_widget:
        display(m)
    
    # Display the map and drawn polygons in Solara
    with solara.Column(style={"min-width": "500px", "height": "500px", "isolation": "isolate"}):
        solara.Text("Drawn Polygons and Coordinates:")
        solara.Text(json.dumps(drawn_polygons.value, indent=2))  # Display drawn polygons with coordinates
        
    # Render the map widget using IPython's display
    display(map_widget)

# Render the component in your Solara notebook
Page()

Cannot show ipywidgets in text

Type: Point
Coordinates: [4.264295, 52.094297]
Type: Point
Coordinates: [4.263442, 52.095929]
Type: Polygon
Coordinates: [[[4.258634, 52.099462], [4.258634, 52.101096], [4.262154, 52.101096], [4.262154, 52.099462], [4.258634, 52.099462]]]
Type: Polygon
Coordinates: [[[4.25275, 52.101623], [4.25275, 52.103943], [4.258932, 52.103943], [4.258932, 52.101623], [4.25275, 52.101623]]]


In [28]:
drawn_polygons.value


[{'type': 'Point', 'coordinates': [4.264295, 52.094297]},
 {'type': 'Point', 'coordinates': [4.263442, 52.095929]},
 {'type': 'Polygon',
  'coordinates': [[[4.258634, 52.099462],
    [4.258634, 52.101096],
    [4.262154, 52.101096],
    [4.262154, 52.099462],
    [4.258634, 52.099462]]]},
 {'type': 'Polygon',
  'coordinates': [[[4.25275, 52.101623],
    [4.25275, 52.103943],
    [4.258932, 52.103943],
    [4.258932, 52.101623],
    [4.25275, 52.101623]]]}]